# Qubitization Walk Operator

In [ ]:
from qualtran import Bloq, CompositeBloq, BloqBuilder, Signature, Register
from qualtran import QBit, QInt, QUInt, QAny
from qualtran.drawing import show_bloq, show_call_graph, show_counts_sigma
from typing import *
import numpy as np
import sympy
import cirq

## `QubitizationWalkOperator`
Constructs a Szegedy Quantum Walk operator using LCU oracles SELECT and PREPARE.

Constructs a Szegedy quantum walk operator $W = R_{L} . SELECT$, which is a product of
two reflections $R_{L} = (2|L><L| - I)$ and $SELECT=\sum_{l}|l><l|H_{l}$.

The action of $W$ partitions the Hilbert space into a direct sum of two-dimensional irreducible
vector spaces. For an arbitrary eigenstate $|k>$ of $H$ with eigenvalue $E_k$, $|\ell>|k>$ and
an orthogonal state $\phi_{k}$ span the irreducible two-dimensional space that $|\ell>|k>$ is
in under the action of $W$. In this space, $W$ implements a Pauli-Y rotation by an angle of
$-2arccos(E_{k} / \lambda)$ s.t. $W = e^{i arccos(E_k / \lambda) Y}$.

Thus, the walk operator $W$ encodes the spectrum of $H$ as a function of eigenphases of $W$
s.t. $spectrum(H) = \lambda cos(arg(spectrum(W)))$ where $arg(e^{i\phi}) = \phi$.

#### Parameters
 - `select`: The SELECT lcu gate implementing $SELECT=\sum_{l}|l><l|H_{l}$.
 - `prepare`: Then PREPARE lcu gate implementing $PREPARE|00...00> = \sum_{l=0}^{L - 1}\sqrt{\frac{w_{l}}{\lambda}} |l> = |\ell>$
 - `control_val`: If 0/1, a controlled version of the walk operator is constructed. Defaults to None, in which case the resulting walk operator is not controlled.
 - `power`: Constructs $W^{power}$ by repeatedly decomposing into `power` copies of $W$. Defaults to 1. 

#### References
 - [Encoding Electronic Spectra in Quantum Circuits with Linear T Complexity] (https://arxiv.org/abs/1805.03662).     Babbush et. al. (2018). Figure 1.


In [ ]:
from qualtran.bloqs.qubitization_walk_operator import QubitizationWalkOperator

### Example Instances

In [ ]:
from qualtran.bloqs.qubitization_walk_operator_test import get_walk_operator_for_1d_ising_model

walk_op = get_walk_operator_for_1d_ising_model(4, 2e-1)

#### Graphical Signature

In [ ]:
from qualtran.drawing import show_bloqs
show_bloqs([walk_op, thc_walk_op],
           ['`walk_op`', '`thc_walk_op`'])

### Call Graph

In [ ]:
from qualtran.resource_counting.generalizers import ignore_split_join
walk_op_g, walk_op_sigma = walk_op.call_graph(max_depth=1, generalizer=ignore_split_join)
show_call_graph(walk_op_g)
show_counts_sigma(walk_op_sigma)

In [ ]:
from openfermion.resource_estimates.utils import QI

from qualtran.bloqs.chemistry.thc.walk_operator import get_walk_operator_for_thc_ham

# Li et al parameters from openfermion.resource_estimates.thc.compute_cost_thc_test
num_spinorb = 152
# lambda_tot = 1201.5
num_bits_state_prep = 10
num_bits_rot = 20
thc_dim = 450
num_spat = num_spinorb // 2
tpq = np.random.normal(size=(num_spat, num_spat))
tpq = 0.5 * (tpq + tpq) / 2
zeta = np.random.normal(size=(thc_dim, thc_dim))
zeta = 0.5 * (zeta + zeta) / 2
qroam_blocking_factor = np.power(2, QI(thc_dim + num_spat)[0])
_thc_walk_op = get_walk_operator_for_thc_ham(
    tpq,
    zeta,
    num_bits_state_prep=num_bits_state_prep,
    num_bits_theta=num_bits_rot,
    kr1=qroam_blocking_factor,
    kr2=qroam_blocking_factor,
)